In [1]:
from localization_scripts.imports import *
%load_ext autoreload
%autoreload 2

In [2]:
filename = '/home/smlm-workstation/event-smlm/our_ev_smlm_recordings/MT_CL_2023-05-31_14-55-59.raw'
if os.path.basename(filename)[-4:] == ".raw":
    events = raw_events_to_array(filename).astype(
        [("x", "uint16"), ("y", "uint16"), ("p", "byte"), ("t", "uint64")]
    )

slice = events[(events["t"] < 200e6)]

In [3]:
from numba import njit, prange, types, typed, jit
from numba.typed import List
# from localization_scripts.event_array_processing import array_to_polarity_map
NUM_CORES = multiprocessing.cpu_count()

"""PROMINENCE is the prominence of the peaks in the convolved signals.
Smaller value detects more peaks, increasing the evaluation time."""
PROMINENCE = 18

"""DATASEET_FWHM is the FWHM of the PSF in the dataset in pixels."""
DATASEET_FWHM = 6

"""PEAK_TIME_THRESHOLD is the maximum time difference between two peaks in order to be considered as the same peak."""
PEAK_TIME_THRESHOLD = 40e3

"""PEAK_NEIGHBORS is the number of neighboring pixels to be considered when filtering same peaks."""
PEAK_NEIGHBORS = 7

"""ROI_RADIUS is the radius of the generated ROI in pixels."""
ROI_RADIUS = 8

events = slice
start_time = time.time()

# Get the minimum and maximum x and y coordinates
min_x = events["x"].min()
min_y = events["y"].min()
max_x = events["x"].max()
max_y = events["y"].max()

# Create coordinate lists
y_coords, x_coords = [min_y, max_y], [min_x, max_x]
coords = generate_coord_lists(y_coords[0], y_coords[1], x_coords[0], x_coords[1])

# Generate dictionaries and calculate max length
print(f"Analyzing the data using {NUM_CORES} cores... Events go brrrrrrrrrrrr!")
print(
    f"Converting events to dictionaries... Elapsed time: {time.time() - start_time:.2f} seconds"
)

dict_events, events_t_p_dict, coords = convert_to_hashmaps(events, filename, max_x, max_y)

Analyzing the data using 24 cores... Events go brrrrrrrrrrrr!
Converting events to dictionaries... Elapsed time: 0.65 seconds


In [6]:
values = list(dict_events.values())

In [11]:
lengths = [val[2] for val in values]

In [12]:
lengths

[ListType[uint64]([138694, ...]),
 ListType[uint64]([171113, ...]),
 ListType[uint64]([901, ...]),
 ListType[uint64]([166, ...]),
 ListType[uint64]([812, ...]),
 ListType[uint64]([305, ...]),
 ListType[uint64]([609, ...]),
 ListType[uint64]([111385, ...]),
 ListType[uint64]([229, ...]),
 ListType[uint64]([484, ...]),
 ListType[uint64]([122780, ...]),
 ListType[uint64]([101363, ...]),
 ListType[uint64]([306, ...]),
 ListType[uint64]([410, ...]),
 ListType[uint64]([89078, ...]),
 ListType[uint64]([155, ...]),
 ListType[uint64]([112556, ...]),
 ListType[uint64]([190, ...]),
 ListType[uint64]([1647, ...]),
 ListType[uint64]([622, ...]),
 ListType[uint64]([816, ...]),
 ListType[uint64]([241, ...]),
 ListType[uint64]([95305, ...]),
 ListType[uint64]([136, ...]),
 ListType[uint64]([202, ...]),
 ListType[uint64]([297, ...]),
 ListType[uint64]([188700, ...]),
 ListType[uint64]([633, ...]),
 ListType[uint64]([314, ...]),
 ListType[uint64]([6352, ...]),
 ListType[uint64]([127, ...]),
 ListType[ui

In [ ]:
from scipy.ndimage import gaussian_filter


plt.figure(figsize=(15, 10))
plt.axis("off")
widefield_filtered = gaussian_filter(widefield, sigma=9.5, radius=9)
useful_pixels = np.where(widefield_filtered >= np.percentile(widefield_filtered, 60), widefield, 0)

# plt.imshow(gaussian_filter(widefield), sigma=0.8, radius=5), cmap="gray", vmax=30)
plt.imshow(np.where(widefield_filtered >= np.percentile(widefield_filtered, 50), widefield, 0), cmap="gray", vmax=100)

In [ ]:
import numpy as np

def find_top_1_percent_values(arr):
    flat_arr = arr.flatten()  # Flatten the 2D array to a 1D array
    cutoff = np.percentile(flat_arr, 99.99)  # Calculate the cutoff value for the top 1%
    
    coordinates = np.argwhere(arr >= cutoff)  # Get the coordinates where values are greater than or equal to the cutoff
    values = arr[coordinates[:, 0], coordinates[:, 1]]  # Get the corresponding values
    
    result = {tuple(coord): value for coord, value in zip(coordinates, values)}  # Create a dictionary with coordinates as keys and values as values
    return result

result = find_top_1_percent_values(widefield)
print(result)